# Basic Python 101

Python is an interpreted language (as opposed to the compiled languages like C), which means that you will be writing and running Python scripts. 

Two of the most common Python data structures are lists and dictionaries. We will cover them briefly here. 

## Lists

Lists are sequences of objects that are denoted by brackets. We can 
* initialize a list by directly specifying their elements
* these elements do not have to be the same

In [1]:
list1 = []
list2 = ['a', 'b', 'c']
list3 = [1, 2, 3 ]
list4 = ["red", "green", "yellow"]
list5 = ["blue", 0.5, 10, 'a']

print list1
print list2
print list3
print list4
print list5

[]
['a', 'b', 'c']
[1, 2, 3]
['red', 'green', 'yellow']
['blue', 0.5, 10, 'a']


We can index into lists using the `[start:stop:step]` or `[start:stop]` notation. Note that not specifying a start or stop defaults to the beginning and end of a list, and step defaults to 1. Lists also take negative indices. 

In [2]:
print list2[:]
print list2[1:]
print list2[:2]
print list2[-1]
print list2[0:3:2]

['a', 'b', 'c']
['b', 'c']
['a', 'b']
c
['a', 'c']


### List comprehensions 

We can also create lists from other lists. One way to do this is by using a for loop, but we can also create them with what are known as list comprehensions.

In [3]:
print [ i*10 for i in list3 ]
print [ len(s) for s in list4 ]

[10, 20, 30]
[3, 5, 6]


We can iterate over lists using `for` loops: 

In [4]:
for s in list4:
    print s

red
green
yellow


Finally, in this example we can import the entirety of Shakespeare's work as a list of lines. 

In [5]:
f = open('shakespeare.txt', 'r')
shakespeare = f.readlines()
print len(shakespeare)
print shakespeare[0:5]

IOError: [Errno 2] No such file or directory: 'shakespeare.txt'

Putting this all together, we can write a short script that counts the total number of words in Shakespeare using a simple list comprehension.

In [ ]:
num_words = sum([len(line.split()) for line in shakespeare])
print num_words

### Generator expressions
Using the same list comprehensions without the brackets results in a generator expression. For example, the following generator expression generates every integer from 1 to `n`.

A generator expression is slightly different from a list in that the contents of entire list are generated on the fly as opposed to all at once. This is particularly useful when you don't need to access the entire list at once, but only need to iterate over over elements. For example, in the previous line where we counted the number of words in Shakespeare, we could have instead omitted the brackets and passed a generator expression to avoid allocating memory for the entire list. 

In [ ]:
print sum(len(line.split( )) for line in shakespeare)

## List Performance

Lists are implemented under the hood as variable length arrays. So when a list runs out of memory, it allocates more memory to extend the array. In fact, it overallocates. This makes indexing a very cheap `O(1)` operation, and appending also amortized `O(1)`. For example, let's try converting the list of Shakespeare lines to a list of words. 

In [ ]:
%%timeit
list_of_words = []
for line in shakespeare[:10000]:
    for word in line.split():
        list_of_words.insert(0, word)

In [ ]:
%%timeit
list_of_words = []
for line in shakespeare[:10000]:
    for word in line.split():
        list_of_words.append(word)

In the first example, we insert the word at the beginning of the list. Since this list is really a variable length array, inserting into the beginning, as you can probably guess, is terribly inefficient. 

In the second example, we append the word to the end of the list. This ends up being more than 100x faster!

The second version can more concisely be written as the following.

In [ ]:
# %%timeit
list_of_words = []
for line in shakespeare:
    list_of_words += line.split()

# Dictionaries

Dictionaries (dict) are sets of key/value pairs Similar to a list, we can

* initialize a dict by specifying its key/value pairs
* keys can be of any hashable type, values can be any type
* can also use a list comprehension-like construction

In [ ]:
dict1 = {}
dict2 = { "blue" : 3, "green" : 4, "red" : 5 }
dict3 = { s[:2] : s[2:] for s in list4 }

print dict1
print dict2
print dict3

We can use a dictionary to count the total number of each word in the Shakespeare text, and print the most and least common words. 

In [ ]:
import string

list_of_words = []
for line in shakespeare:
    list_of_words += line.split()
    
processed_list_of_words = [w.lower().translate(string.maketrans("",""), string.punctuation) for w in list_of_words]

def count(words):
    d = {}
    for word in words:
        if word in d.keys(): 
            d[word] += 1
        else:
            d[word] = 1
    return d

# takes too long!
# d = count(processed_list_of_words)
d = count(processed_list_of_words[:10000])
        
print sorted(d, key=d.get)[:5]
print sorted(d, key=d.get, reverse=True)[:5]

### Dictionary performance
Most of the performance improvements for dictionaries come from not needing to construct or iterate through all the keys or values. As a result, it is important to avoid creating lists of keys or lists of values when possible. Do you see the problem with the above implementation for `count`?

In [ ]:
%timeit d = count(processed_list_of_words[:10000])

The above code is creating a list of keys on every iteration with `d.keys()`! Since the `in` keyword is overloaded for dictionaries, it is much more efficient to use the following instead to check for dictionary membership. 

In [ ]:
def faster_count(words):
    d = {}
    for word in words:
        if word in d: 
            d[word] += 1
        else:
            d[word] = 1
    return d

%timeit d = faster_count(processed_list_of_words[:10000])

In [ ]:
word_counts = faster_count(processed_list_of_words)

When do you end up wanting to iterate through a dictionary, you still want to avoid creating lists. Use `iterkeys` and `itervalues` to create generator expression instead of lists, which will be more efficient on larger dictionaries. Let's now create an example test dictionary and run some timing calls. 

In [ ]:
speed_test = dict(zip(xrange(1000000),xrange(1000000)))

In [ ]:
%timeit for k in speed_test.keys(): pass

In [ ]:
%timeit for k in speed_test.iterkeys(): pass

We see that there is almost a two times slowdown when creating the list of keys as opposed to using an iterator. That being said, if you find yourself continuously iterating over keys and have memory to spare, it may be worth it to take the initial computation cost to create the keys, then reuse the list for fast iteration. 

#### Note for the Python 3 folks
Python 2's `iterkeys` and `itervalues` are equivalent to Python 3's `keys` and `values` methods. The original Python 2 `keys` and `values` have been backported to `viewkeys` and `viewvalues` in Python 3. 

# Classes

Next we'll go through the structure of a basic class. Classes in Python are similar to those of other languages, and define a new namespace with a local scope (so functions and variables defined in a class are in this local namespace). Classes have two usages: attribute references and instantiation. 

### Attributes (variables)
A class can have variables or functions, which are typically referred to as class attributes. These can be accessed with the notation of `ClassName.attributeName`, so in the above example we can access all the attributes as follows:

In [ ]:
class ClassName:
    variable1 = "value1"
    variable2 = "value2"
    
    # ...

print ClassName.variable1, ClassName.variable2

### Attributes (functions)
A class can define functions as attributes as well. By default, all functions take as their first argument a variable that represents the instance of the class Object that is calling the function. Note that this means we have to instantiate an object of the class before we can call the class functions. 

In [ ]:
class ClassName:
    variable1 = "value1"
    variable2 = "value2"
    
    def f(self):
        return self
    
    def addVariable1(self, arg1):
        return arg1 + self.variable1
    
    def setVariable2(self,s):
        self.variable2 = s
    
a = ClassName()
print a.variable1
print a.f()
print a.addVariable1("string")
a.setVariable2("newValue")
print a.variable2

Calling a class function directly from its class name without instantiating a class instance will throw an error, unless you pass a variable it can use as `self`. 

In [ ]:
ClassName.f()

In [ ]:
ClassName.f(a)

We can add decorators to functions that change the call signatures of the class functions. 
* @classmethod will allow the first argument to be the class itself instead of a class instance
* @staticmethod will remove the first argument entirely

In [ ]:
class ClassName:
    variable1 = "value1"
    variable2 = "value2"
    
    def f(self,x):
        return (self,x)
    
    @classmethod
    def class_f(cls, x):
        return (cls, x)
    
    @staticmethod
    def static_f(x):
        return x
    
a = ClassName()
print a.f("string")
print ClassName.class_f("string")
print a.static_f("string")

### Naming conventions and special functions
In python, there are no truly private variables. Instead, there are some naming conventions that utilize the underscore character. 
* a single leading underscore is for weak private use (import * will ignore these)
* a double leading underscore is for stronger private use (will result in name mangling outside of the class)
* a double leading underscore and trailing underscore is for "magic" objects or attributes

#### Magic functions
Magic functions have specific use cases and should only be used as documented. Some of these include
* `__init__(self, variable1, variable2, ...)` defines the class initialization function
* `__str__(self)` defines the string that is printed when an instance is passed to `print()`
* `__del__(self)` defines what happens when an instance is about to be destroyed
* you can override many other magic functions (like getters and setters for bracket notation) and much more, see https://docs.python.org/2/reference/datamodel.html

In [ ]:
class ClassName:
    def __init__(self, s):
        self.variable = s
    
    def __str__(self):
        return "ClassName('"+self.variable+"')"
    
a = ClassName("foo")
print a

Putting this all together, we can create a TextReader class for our Shakespeare code. 

In [ ]:
class TextReader:
    def __init__(self, fname):
        f = open(fname, 'r')
        lines = f.readlines()
        
        self.words = []
        for line in lines:
            self.words += line.split()
    
    def word_counts(self):
        d = {}
        for word in self.words:
            if word in d: 
                d[word] += 1
            else:
                d[word] = 1
        return d

t = TextReader('shakespeare.txt')
d = t.word_counts()
print sorted(d, key=d.get)[:5]
print sorted(d, key=d.get, reverse=True)[:5]

# Basic Visualizations with matplotlib

One of the things you should always do before you try to run any algorithm is to inspect your data. In Jupyter notebook, it is possible to create matplotlib graphs inline inside the notebook. 

A good tutorial on plotting using matplotlib in python is here http://matplotlib.org/users/pyplot_tutorial.html.

You will need to run the following magic command and import matplotlib (already included in Anaconda) to make plots show up in the Jupyter notebook. 

In [ ]:
import matplotlib
# Use svg backend for better quality
matplotlib.use("svg")
%matplotlib inline

import matplotlib.pyplot as plt
# Default matplotlib coloring is really ugly
plt.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = (10.0, 5.0)

The usage of `matplotlib` in Python is very similar to that of Matlab. You can make your typical line plots, like the following sin and cosine curve. 

In [ ]:
import numpy as np
X = np.linspace(-np.pi,np.pi,256)
Y_cos = np.cos(X)
Y_sin = np.sin(X)
plt.plot(X, Y_cos)
plt.plot(X, Y_sin)
plt.show()

### Shakespeare word counts and histograms

Let's go back to the Shakespeare demo and make a histogram of the word frequencies. 

In [ ]:
n, bins, patches = plt.hist(word_counts.values())
plt.show()

This doesn't look quite so well. We should add a log scale to the y-axis to make this more readable. 

In [ ]:
n, bins, patches = plt.hist(word_counts.values(), log=True)
plt.show()

We immediately see two things from this skewed distribution: 1) almost all words have a small frequency and 2) a handful of words are showing up extremely frequently. What words occur extremely often? How many words occur just once? 

In [ ]:
sorted_words = sorted(word_counts, key=d.get, reverse=True)
print [(word,word_counts[word]) for word in sorted_words[:5]]

print len([word for word in word_counts if word_counts[word] == 1])

If we make both axes a log scale, the histogram turns out to have a pretty linear shape. 

In [ ]:
n, bins, patches = plt.hist(word_counts.values(), log=True, bins=np.logspace(1,4,10))
plt.gca().set_xscale("log")
plt.show()

Just for fun, we can create a word cloud of the Shakespearean text. You can try this at home on your own text data, using 
```
pip install wordcloud
``` 
Note that this package contains a list of 'stopwords' that they don't count, so the most frequent words we found before will not show up. 

In [ ]:
from wordcloud import WordCloud
# wc = WordCloud().generate(open('shakespeare.txt', 'r').read())
wc = WordCloud().generate(" ".join(processed_list_of_words))
plt.imshow(wc)
plt.axis("off")

### Piazza statistics (multiple figures, scatter plots, layering plots)
Next we'll do a few visualization with the Piazza statistics from the course Piazza. 

In [ ]:
import pandas as pd
df = pd.DataFrame.from_csv('piazza.csv')

# Removing the names and emails
df= df.reset_index()
df = df[['days online', 'views', 'contributions', 'questions', 'notes', 'answers']]
print df.head()

# If you didn't want to use pandas you could read in the csv manually with 
#
# output = []
# with open('piazza.csv', 'rb') as csvfile:
#     reader = csv.reader(csvfile, delimiter=',')
#     row_labels = next(reader,None)[3:]
#     for row in reader:
#         output.append(row[3:])
# data = np.array(output)


With matplotlib, you can make multiple plots in the same figure using subplots. 

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
plt.figure(1)
plt.gca().set_xscale("linear")
i = 1
for column in df:
    plt.subplot(320 + i)
    plt.hist(df[column])
    plt.xlabel(column)
    i += 1

In addition to histograms, you can also make scatter plots. Here we plot the number of student contributions against all other features. 

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
plt.figure(1)
plt.gca().set_xscale("linear")
i = 1
for column in df:
    plt.subplot(320 + i)
    plt.scatter(df['contributions'],df[column])
    plt.xlabel('contributions')
    plt.ylabel(column)
    i += 1

Some scientific computing tools have functions that will perform common visualizations of your data. One example is pair-wise scatter plots with Pandas, which will also create the histograms. While these functions are convenient for initial exploration, in the end you will probably want to write your own visualizations that fit the data you're using and highlight what you're trying to convey. 

In [ ]:
matplotlib.rcParams['figure.figsize'] = (13.0, 13.0)
axes = pd.tools.plotting.scatter_matrix(df)
plt.tight_layout()
plt.show()

From this data you can see that there are a number of extreme data points. This is because I left in the instructors :) removing the instructors and recreating the plots, we see the following. 

In [ ]:
student_df = df[4:]
matplotlib.rcParams['figure.figsize'] = (13.0, 13.0)
axes = pd.tools.plotting.scatter_matrix(student_df)
plt.tight_layout()
plt.show()

We can also overlay plots on other plots. For example, if we fit an geometric distribution to the number of student contributions (which has the somewhat amusing connotation that every student has a probability (1-p) of posting again and p chance of never posting again), we get the following: 

In [ ]:
from scipy.stats import geom
X = np.linspace(1,20,20)
p = 1./np.mean(student_df['contributions'])
y = geom.pmf(X,p)
n = len(student_df['contributions'])
plt.plot(X,y, 'o', markersize=10)
plt.hist(student_df['contributions'], weights=np.ones(n)/n)
plt.show()

print np.mean(student_df[student_df['contributions']!=0])

# possibily interesting statistic for the class
# DISCLAIMER: this is not related to how the actual cutoff will be determined :)
variance = (1-p)/(p*p)
std = np.sqrt(variance)
print 1./p - std

### Matrix visualizations (heatmaps)

Here, we will cover a simple way to visualize data in a matrix. One common way to view matrix data is via a heat map, which translates matrix values to various shades of colors. We can visualize the covariance matrix of the Piazza data in the following manner. 

In [ ]:
print df.cov()
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
im = plt.imshow(df.cov(), cmap='hot', interpolation='nearest')
r = np.arange(0,6,1)
ax.set_xticks(r)
ax.set_xticklabels(df.columns)
ax.set_yticks(r)
ax.set_yticklabels(df.columns)
ax.yaxis.grid(False)
ax.xaxis.grid(False)
plt.show()